In [1]:
%load_ext autotime

time: 0 ns (started: 2021-07-14 22:15:44 +05:30)


In [2]:
import os
# change working directory

print('Before chdir:', os.getcwd())

os.chdir('F:\\NLP\\intent') 
         
print('After chdir:', os.getcwd())

Before chdir: C:\Users\Ravi
After chdir: F:\NLP\intent
time: 0 ns (started: 2021-07-14 22:15:44 +05:30)


#### Required packages

In [3]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import nltk
from nltk import word_tokenize 
from nltk.util import ngrams
from nltk.collocations import *
from nltk.corpus import stopwords
from sparse_dot_topn import awesome_cossim_topn
from scipy.sparse import csr_matrix
from typing import List
import yake
import traceback
import sys

# import spacy
# nlp = spacy.load("en_core_web_sm")

#Gensim
import gensim
import gensim.corpora as corpora
from gensim.models import CoherenceModel

import warnings
warnings.filterwarnings("ignore")

time: 14.8 s (started: 2021-07-14 22:15:44 +05:30)


#### Required Functions

In [4]:
# Preprocessing function 1
import re
import string

def process_text(sentence):
    
    '''Make text lowercase, remove text in square brackets,remove links,remove punctuation
    and remove words containing numbers.'''
    #lower case 
    sentence = sentence.lower()
    # square brackets
    sentence = re.sub('\[', ' ', sentence)
    sentence = re.sub(']', ' ', sentence)
    # curly brackets
    sentence = re.sub('\{', ' ', sentence)
    sentence = re.sub('}', ' ', sentence)
    # square brackets
#     sentence = re.sub('\[.*?\]', '', sentence)  
    # hyperlinks
    sentence = re.sub('https?://\S+|www\.\S+', '', sentence)
    sentence = re.sub('<.*?>+', '', sentence)
    # remove punctuation
    sentence = re.sub('[%s]' % re.escape(string.punctuation), '', sentence)
    sentence = re.sub('\n', '', sentence)
    # remove year
    sentence = re.sub(r"\b(19[40][0-9]|20[0-1][0-9]|2020)\b",'',sentence)
    # remove month names
    mp = r"(\b\d{1,2}\D{0,3})?\b(?:Jan|jan(?:uary)?|Feb|feb(?:ruary)?|Mar|mar(?:ch)?|Apr|mar(?:il)?|May|may|Jun|jun(?:e)?|Jul|jul(?:y)?|Aug|aug(?:ust)?|Sep|sep(?:tember)?|Oct|oct(?:ober)?|(Nov|nov|Dec|dec)(?:ember)?)\D?(\d{1,2}(st|nd|rd|th)?)?(([,.\-\/])\D?)?((19[7-9]\d|20\d{2})|\d{2})*"
    sentence = re.sub(mp, '', sentence)
     
    # remove words containing numbers
    sentence = re.sub('\w*\d\w*', '', sentence)
    # remove numbers 
    sentence = re.sub(r'[0-9]+','',sentence)
    sentence = " ".join(sentence.split())
    
    return sentence

time: 0 ns (started: 2021-07-14 22:15:59 +05:30)


In [5]:
# Preprocessing function 2
import pandas as pd
import numpy as np
import dask.dataframe as dd
import nltk
import en_core_web_sm
from bs4 import BeautifulSoup
import re
import unicodedata
from contractions import CONTRACTION_MAP
from nltk.tokenize.toktok import ToktokTokenizer
tokenizer = ToktokTokenizer()
stopword_list = nltk.corpus.stopwords.words('english')
from nltk.corpus import words
engwords = words.words()
nlp = en_core_web_sm.load()
import warnings
warnings.filterwarnings('ignore')
#Load the custom stopword file
# -------------------------------------------------------------------------------------------------------------------------
custok = []
with open('stopwords.txt', 'r') as f:
    for word in f:
        word = word.split('\n')
        custok.append(word[0])
# -------------------------------------------------------------------------------------------------------------------------
#Define the Schema
dtypes ={
    'Description': np.str
}


def strip_html_tags(text):
    soup = BeautifulSoup(text, "html.parser")
    if bool(soup.find()):
        [s.extract() for s in soup(['iframe', 'script'])]
        stripped_text = soup.get_text()
        stripped_text = re.sub(r'[\r|\n|\r\n]+', '\n', stripped_text)
        stripped_text = re.sub(r'''(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))''', " ", stripped_text)    
    else:
        stripped_text = text
    return stripped_text

def remove_accented_chars(text):
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return text

def remove_special_characters(text, remove_digits=False):
    pattern = r'[^a-zA-Z0-9\s]|\[|\]' if not remove_digits else r'[^a-zA-Z\s]|\[|\]'
    text = re.sub(pattern, '', text)
    # print('removal special characters completed')
    return text

def expand_contractions(text, contraction_mapping=CONTRACTION_MAP):
    
    contractions_pattern = re.compile('({})'.format('|'.join(contraction_mapping.keys())), 
                                      flags=re.IGNORECASE|re.DOTALL)
    def expand_match(contraction):
        match = contraction.group(0)
        first_char = match[0]
        expanded_contraction = contraction_mapping.get(match)\
                                if contraction_mapping.get(match)\
                                else contraction_mapping.get(match.lower())                       
        expanded_contraction = first_char+expanded_contraction[1:]
        return expanded_contraction
        
    expanded_text = contractions_pattern.sub(expand_match, text)
    expanded_text = re.sub("'", "", expanded_text)
    return expanded_text

def remove_stopwords(text, is_lower_case=False, stopwords = stopword_list):
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    if is_lower_case:
        filtered_tokens = [token for token in tokens if token not in stopwords]
    else:
        filtered_tokens = [token for token in tokens if token.lower() not in stopwords]
    filtered_text = ' '.join(filtered_tokens)    
    return filtered_text

def custom_stopwords(text, custok=custok):
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    filtered_custokens = [token for token in tokens if token not in custok]
    filtered_text = ' '.join(filtered_custokens) 
    return filtered_text

def get_keywords(text, eng_words = engwords):
    tokens = tokenizer.tokenize(text)
    eng_tokens = [token for token in tokens if token in eng_words]
    eng_text = ' '.join(eng_tokens)    
    return eng_text

def simple_porter_stemming(text):
    ps = nltk.porter.PorterStemmer()
    text = ' '.join([ps.stem(word) for word in text.split()])   
    return text

def lemmatize_text(text):
    text = nlp(text)
    text = ' '.join([word.lemma_ if word.lemma_ != '-PRON-' else word.text for word in text])
    return text

def remove_repeated_words(text):
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    seen = set()
    seen_add = seen.add

    def add(x):
        seen_add(x)  
        return x
    text = ' '.join(add(i) for i in tokens if i not in seen)
    return text


def clean_text(pDf,pcol):
    pDf = pDf.dropna(subset = [pcol])
    pDf[pcol] = pDf[pcol].apply(lambda s: str(s))
    pDf['Sample'] = pDf[pcol].map(lambda s: s.lower()).map(strip_html_tags).map(remove_special_characters).map(expand_contractions).map(remove_stopwords).map(custom_stopwords).map(process_text)
    return pDf


time: 3.11 s (started: 2021-07-14 22:15:59 +05:30)


In [6]:
# Function for cosine similarity and ngram generation

def cosine_similarity(from_vector: np.ndarray,
                      to_vector: np.ndarray,
                      from_list: List[str],
                      to_list: List[str],
                      to_key_list: List[str],
                      nbest,
                      min_similarity: float = 0) -> pd.DataFrame:
    
    if nbest != None:
        if int(nbest) >  len(to_list):
            raise ValueError('best choice must be less than to_list')
    else:
        nbest = int(1)

    if isinstance(to_vector, np.ndarray):
        to_vector = csr_matrix(to_vector)
    if isinstance(from_vector, np.ndarray):
        from_vector = csr_matrix(from_vector)

    # There is a bug with awesome_cossim_topn that when to_vector and from_vector
    # have the same shape, setting topn to 1 does not work. Apparently, you need
    # to it at least to 2 for it to work

    if int(nbest) <= 1:
        similarity_matrix = awesome_cossim_topn(from_vector, to_vector.T, 2, min_similarity)
    elif int(nbest) > 1:
        similarity_matrix = awesome_cossim_topn(from_vector, to_vector.T, nbest, min_similarity)

    if from_list == to_list:
        similarity_matrix = similarity_matrix.tolil()
        similarity_matrix.setdiag(0.)
        similarity_matrix = similarity_matrix.tocsr()

    if int(nbest) <= 1:
        indices = np.array(similarity_matrix.argmax(axis=1).T).flatten()
        similarity = similarity_matrix.max(axis=1).toarray().T.flatten()
    elif int(nbest) > 1:
        similarity = np.flip(np.take_along_axis(similarity_matrix.toarray(), np.argsort(similarity_matrix.toarray(), axis =1), axis=1) [:,-int(nbest):], axis = 1)
        indices = np.flip(np.argsort(np.array(similarity_matrix.toarray()), axis =1)[:,-int(nbest):], axis = 1)
            
    
    if int(nbest) <= 1:
        matches = [to_list[idx] for idx in indices.flatten()]
        key_matches = [to_key_list[idx] for idx in indices.flatten()]
        matches = pd.DataFrame(np.vstack((from_list, matches, key_matches, similarity)).T, columns=["From", "To", "Key", "Similarity"])
        matches.Similarity = matches.Similarity.astype(float)
        matches.loc[matches.Similarity < 0.001, "To"] = None
        matches.loc[matches.Similarity < 0.001, "Key"] = None
    else:
        matches = [np.array([to_list[idx] for idx in l]) for l in indices] ##In progress
        key_matches = [np.array([to_key_list[idx] for idx in l]) for l in indices] ##In progress
        column = []
        column.append("To")
        for i in range(int(nbest) - 1):
            column.append("BestMatch" + "__" + str(i+1))
        column.append("Key")
        for j in range(int(nbest) - 1):
            column.append("Key" + "__" + str(j+1))
        column.append("Similarity")
        for j in range(int(nbest) - 1):
            column.append("Similarity" + "__" + str(j+1))
            
        matches = pd.concat([pd.DataFrame({'From' : from_list}), pd.DataFrame(np.hstack((matches, key_matches, similarity)), columns= column)], axis =1)
        matches.Similarity = matches.Similarity.astype(float)
        matches.loc[matches.Similarity < 0.001, "To"] = None
        matches.loc[matches.Similarity < 0.001, "Key"] = None
        for i in range(int(nbest) - 1):
            matches.loc[matches.Similarity < 0.001, "BestMatch" + "__" + str(i+1)] = None
            matches.loc[matches.Similarity < 0.001, "Key" + "__" + str(i+1)] = None
        
    return matches


def _create_ngrams(string: str) -> List[str]:
    n_gram_range=(3, 3)
    string = _clean_string(string)
    result = []
    for n in range(n_gram_range[0], n_gram_range[1]+1):
        ngrams = zip(*[string[i:] for i in range(n)])
        ngrams = [''.join(ngram) for ngram in ngrams if ' ' not in ngram]
        result.extend(ngrams)
    return result


def _clean_string(string: str) -> str:
    """ Only keep alphanumerical characters """
    string = re.sub(r'[^A-Za-z0-9 ]+', '', string.lower())
    string = re.sub('\s+', ' ', string).strip()
    return string

time: 15 ms (started: 2021-07-14 22:16:02 +05:30)


In [7]:
def getIntent(pTopicData, pQueryData, pKeyCol, pDescCol, pTopicDesc, pTktno, nbest = 1):
    try:
        #Get the input parameters
        listOfDataFrames = [] 
        pTopicData = pTopicData.dropna(subset = [pTopicDesc])
        pQueryData = pQueryData.dropna(subset = [pDescCol])
        to_list = pQueryData[pDescCol].values.astype('U').tolist()
        Train_CC = pQueryDahi ta[pKeyCol].values.astype('U').tolist()
        keyNames = pTopicData[pTopicDesc].values.astype('U').tolist()
        tktno = pTopicData[pTktno].values
        nbest = int(nbest)
        for item, n in zip(keyNames, tktno):
            from_list = [item]
            if len(to_list) < nbest:
                continue
            vectorizer = TfidfVectorizer(min_df=1, analyzer=_create_ngrams).fit(to_list + from_list)
            X = vectorizer.transform(to_list)
            Y = vectorizer.transform(from_list)
            to_vector = csr_matrix(X)
            from_vector = csr_matrix(Y)
            matches = cosine_similarity(Y, X, from_list, to_list, Train_CC, nbest)
            tempDF = pd.DataFrame()
            tempDF[pKeyCol] = matches.iloc[:,nbest+1:2*nbest+1].values.tolist()[-1]
            tempDF[pTktno] = nbest*[n]    
            tempDF['similarity_index_score'] = [float(x) for x in matches.iloc[:,2*nbest+1:].values.tolist()[-1]]
            listOfDataFrames.append(tempDF)
        result = pd.concat(listOfDataFrames)
        
#         merged = pd.merge(result, pTopicData, how = 'inner', on = pTktno )
#         merged = merged.drop_duplicates(subset = pTktno)
# #         
#         merged['topic_confidence_level'] = merged.similarity_index_score.values.tolist()
#         merged['topic'] = merged[pKeyCol].values.tolist()
#         merged['recommendation'] = [('High' if i > 0.3 else ('Low' if i < 0.15 else 'Medium')) for i in merged['topic_confidence_level']]

        return result
    except Exception as e:
        print(traceback.format_exc(), e)
        return (-1)

time: 94 ms (started: 2021-07-14 22:16:02 +05:30)


In [15]:
def getIntentWord(pTopicData, pQueryData, pKeyCol, pDescCol, pTopicDesc, pTktno, nbest = 1):
    try:
        #Get the input parameters
        listOfDataFrames = [] 
        pTopicData = pTopicData.dropna(subset = [pTopicDesc])
        pQueryData = pQueryData.dropna(subset = [pDescCol])
        to_list = pQueryData[pDescCol].values.astype('U').tolist()
        Train_CC = pQueryData[pKeyCol].values.astype('U').tolist()
        keyNames = pTopicData[pTopicDesc].values.astype('U').tolist()
        tktno = pTopicData[pTktno].values
        nbest = int(nbest)
        for item, n in zip(keyNames, tktno):
            from_list = [item]
            if len(to_list) < nbest:
                continue
            vectorizer = TfidfVectorizer(min_df=1, analyzer='word').fit(to_list + from_list)
            X = vectorizer.transform(to_list)
            Y = vectorizer.transform(from_list)
            to_vector = csr_matrix(X)
            from_vector = csr_matrix(Y)
            matches = cosine_similarity(Y, X, from_list, to_list, Train_CC, nbest)
            tempDF = pd.DataFrame()
            tempDF[pKeyCol] = matches.iloc[:,nbest+1:2*nbest+1].values.tolist()[-1]
            tempDF[pTktno] = nbest*[n]    
            tempDF['similarity_index_score'] = [float(x) for x in matches.iloc[:,2*nbest+1:].values.tolist()[-1]]
            listOfDataFrames.append(tempDF)
        result = pd.concat(listOfDataFrames)
        
#         merged = pd.merge(result, pTopicData, how = 'inner', on = pTktno )
#         merged = merged.drop_duplicates(subset = pTktno)
# #         
#         merged['topic_confidence_level'] = merged.similarity_index_score.values.tolist()
#         merged['topic'] = merged[pKeyCol].values.tolist()
#         merged['recommendation'] = [('High' if i > 0.3 else ('Low' if i < 0.15 else 'Medium')) for i in merged['topic_confidence_level']]

        return result
    except Exception as e:
        print(traceback.format_exc(), e)
        return (-1)

time: 0 ns (started: 2021-07-15 00:24:33 +05:30)


#### Step1: Read intent training data

In [8]:
qdf = pd.read_excel('Intent_training_data3A_tokens.xlsx')

print(qdf.shape)
qdf.head()

(34650, 7)


,sample,level1,level2,Sample,lvl12,train_ykw,train_tfidf_kw
0,"hello, user can not log in to 10stream, could...",access issue,unable to login,user log check account,access issue__unable to login,account log check user,check log account user
1,lokesh desai called us and said that he does n...,access issue,other,lokesh desai called access inform form,access issue__other,lokesh form desai access inform called,desai lokesh inform form called access
2,this is the message that appears when im tryin...,access issue,unable to login,message appears log session didnt aminambresdr...,access issue__unable to login,start account include browser sign automatical...,sign aminambresdrivcom site time httpstennecos...
3,she contacted us on behalf of the vice preside...,access issue,other,contacted behalf vice president department iss...,access issue__other,contacted confirmed mentioned vice informed de...,connected zscaler vice president behalf mentio...
4,the access to folder on \\bhsn-file\ece is not...,access issue,other,access folder bhsnfileece working folders,access issue__other,access folder bhsnfileece working folders,bhsnfileece folders folder working access


time: 6.67 s (started: 2021-07-14 22:16:02 +05:30)


In [31]:
qdf1 = qdf.dropna(subset = ['train_ykw'])
print(qdf1.shape)
qdf1.head()

(33946, 7)


,sample,level1,level2,Sample,lvl12,train_ykw,train_tfidf_kw
0,"hello, user can not log in to 10stream, could...",access issue,unable to login,user log check account,access issue__unable to login,account log check user,check log account user
1,lokesh desai called us and said that he does n...,access issue,other,lokesh desai called access inform form,access issue__other,lokesh form desai access inform called,desai lokesh inform form called access
2,this is the message that appears when im tryin...,access issue,unable to login,message appears log session didnt aminambresdr...,access issue__unable to login,start account include browser sign automatical...,sign aminambresdrivcom site time httpstennecos...
3,she contacted us on behalf of the vice preside...,access issue,other,contacted behalf vice president department iss...,access issue__other,contacted confirmed mentioned vice informed de...,connected zscaler vice president behalf mentio...
4,the access to folder on \\bhsn-file\ece is not...,access issue,other,access folder bhsnfileece working folders,access issue__other,access folder bhsnfileece working folders,bhsnfileece folders folder working access


time: 31 ms (started: 2021-07-15 08:58:15 +05:30)


In [32]:
qdf2 = qdf1.groupby(['lvl12'])['train_ykw'].agg(' '.join).reset_index(name = 'comb_desc')
print(qdf2.shape)
qdf2.head()

(318, 2)


,lvl12,comb_desc
0,access issue__account locked,account locked windows user account message er...
1,access issue__authentication error,phone reach user message authenticate asked an...
2,access issue__authorization issue,failed authentication failed authentication cl...
3,access issue__login issue,login user description sujesh kannsuje kannan ...
4,access issue__miscellaneous,access tms issue entitlements accessing proble...


time: 31 ms (started: 2021-07-15 08:58:23 +05:30)


#### Step5: Get keywords for decription grouped by intents

In [11]:
tdf = pd.read_excel('topicdata_preprocessed_V1.xlsx')
print(tdf.shape)
tdf.head()

(32355, 12)


,ticket_no,ticket_type,ticket_description,preprocessed_description,solvedworkgroup,topic,uploadid,topic_confidence_level,action_flag,action_status,split_number,Sample
0,21983985,Service Request,mPOS: PR 14314 - Transactions not posting / Mi...,post miss,SAP BW AS,post__remorse__reprint__variance,6,0.0673,A,Initial,0,mpos transactions posting missing transactions
1,21983959,Incident,Quikview: Transaction did not post to customer...,transaction post account,SAP BW AS,retail__store__account__transaction,6,0.0702,A,Initial,0,quikview transaction post customers account
2,21982340,Incident,Retail Inventory discrepancy / Issue (6%),retail inventory discrepancy,SAP BW AS,discrepancy__signature__rebellion__customer,6,0.0660,A,Initial,0,retail inventory discrepancy issue
3,21991703,Incident,Retail Inventory discrepency / Issue (6%),retail inventory,SAP BW AS,inventory__pier__rebellion__customer,6,0.0545,A,Initial,0,retail inventory discrepency issue
4,21992515,Incident,RE: SAP 2940 - Chicago Signature Store - Missi...,signature store miss transaction pier,SAP BW AS,retail__store__account__transaction,6,0.0717,A,Initial,0,chicago signature store missing transaction pier


time: 9.39 s (started: 2021-07-14 22:16:09 +05:30)


In [33]:
dfm = getIntentWord(tdf, qdf2, 'lvl12','comb_desc', 'Sample', 'ticket_no', nbest = 1)
dfm

,lvl12,ticket_no,similarity_index_score
0,transaction issue__transaction error,21983985,0.520172
0,transaction request__miscellaneous,21983959,0.241016
0,inventory issue__inventory revaluation request,21982340,0.389572
0,inventory issue__inventory revaluation request,21991703,0.372701
0,missing data__miscellaneous,21992515,0.243764
...,...,...,...
0,update request__miscellaneous,AL-1245,0.276111
0,update request__miscellaneous,SL-1058,0.227253
0,update request__miscellaneous,TL3-13736,0.315714
0,adp request__registeration code,AL-1231,0.097757


time: 2h 42min 16s (started: 2021-07-15 08:58:42 +05:30)


In [ ]:
# dfm.to_excel("getIntent_V1.xlsx", index = False)

In [34]:
dfm1 = dfm.drop_duplicates(subset ="ticket_no")

time: 31 ms (started: 2021-07-15 11:48:03 +05:30)


In [36]:
pd.merge(dfm1, tdf,  how='inner', on='ticket_no')

,lvl12,ticket_no,similarity_index_score,ticket_type,ticket_description,preprocessed_description,solvedworkgroup,topic,uploadid,topic_confidence_level,action_flag,action_status,split_number,Sample
0,transaction issue__transaction error,21983985,0.520172,Service Request,mPOS: PR 14314 - Transactions not posting / Mi...,post miss,SAP BW AS,post__remorse__reprint__variance,6,0.0673,A,Initial,0,mpos transactions posting missing transactions
1,transaction issue__transaction error,21983985,0.520172,Service Request,mPOS: PR 14314 - Transactions not posting / Mi...,post miss,SAP BW AS,post__complete__posting__customer,7,0.0652,A,Initial,0,mpos transactions posting missing transactions
2,transaction issue__transaction error,21983985,0.520172,Service Request,mPOS: PR 14314 - Transactions not posting / Mi...,post miss,SAP BW AS,post__complete__posting__customer,2,0.0652,A,Initial,0,mpos transactions posting missing transactions
3,transaction issue__transaction error,21983985,0.520172,Service Request,mPOS: PR 14314 - Transactions not posting / Mi...,post miss,SAP BW AS,post__unable__account__store,3,0.0673,A,Initial,0,mpos transactions posting missing transactions
4,transaction issue__transaction error,21983985,0.520172,Service Request,mPOS: PR 14314 - Transactions not posting / Mi...,post miss,SAP BW AS,post__count__store__variance,4,0.0673,A,Initial,0,mpos transactions posting missing transactions
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32272,update request__miscellaneous,AL-1245,0.276111,Service Request,update non-prod credentials to be different th...,update prod,Media Library App Support,update__compass__module__level,32,0.0673,A,Initial,0,update nonprod credentials prod
32273,update request__miscellaneous,SL-1058,0.227253,Service Request,Update non-prod DB credentials (Application/Wi...,update job,Media Library App Support,update__compass__module__level,32,0.0652,A,Initial,0,update nonprod db credentials applicationwildf...
32274,update request__miscellaneous,TL3-13736,0.315714,Service Request,"Update profile information from Salesforce, no...",update profile,Media Library App Support,code__profile__series__deployment,32,0.0630,A,Initial,0,update profile salesforce
32275,adp request__registeration code,AL-1231,0.097757,Service Request,Veracode: Use of Externally-Controlled Input t...,input select class code,Media Library App Support,datum__local__functionality__assign,32,0.0648,A,Initial,0,veracode externallycontrolled input select cla...


time: 78 ms (started: 2021-07-15 11:48:47 +05:30)


In [ ]:
dfw = getIntentWord(tdf, qdf2, 'lvl12','comb_desc', 'Sample', 'ticket_no', nbest = 1)
dfw